# Pipeline for AutoML Inference

Azure Machine Learning Pipeline を利用して、再利用可能なパイプラインを作成することができます。本 Notebook では AutoML で構築したモデルの推論のパイプラインを作成します。

# 1. 事前準備

## 1.1 Python SDK のインポート
Azure Machine Learning の Python SDK などをインポートします。

In [ ]:
import pandas as pd
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core import Environment

Azure ML Python SDK のバージョンが 1.8.0 以上になっていることを確認します。

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

## 1.2 Azure ML Workspace との接続

In [ ]:
ws = Workspace.from_config()

In [ ]:
experiment_name = 'livedoor-news-classification-BERT-pipeline-inference'
experiment = Experiment(ws, experiment_name)

output = {}
#output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## 1.3 計算環境の準備
BERT を利用するための GPU の `Compute Cluster` を準備します。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## 1.4 推論データの準備
今回は **livedoor ニュースのコーパス** を推論データとして利用しています。適宜変更してください。

In [ ]:
inference_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text"])
inference_dataset.take(5).to_pandas_dataframe()

## 1.5 Environment の準備
Azure Machine Learning Enviroment の準備をします。ここでは Curated を使いますが、適宜変更してください。

In [ ]:
envs = Environment.list(workspace=ws)
# # curated の Environment の一覧を出力
#[env if env.startswith("AzureML") else None for env in envs]

In [ ]:
env =  Environment.get(workspace=ws, name='AzureML-AutoML-DNN-GPU')

In [ ]:
# 推論時に利用する Environment として指定します。
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment = env

# 2. 機械学習パイプライン

## 2.1 機械学習パイプラインの作成

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

ds_input = inference_dataset.as_named_input('input1')

project_folder = '../code/deployment'

# 推論スクリプトを実行するStep
inferenceStep = PythonScriptStep(
    script_name="inference_script.py",
    inputs=[ds_input],
    compute_target=compute_target,
    source_directory=project_folder,
    runconfig=run_config,
    allow_reuse=True
)

In [ ]:
from azureml.pipeline.core import Pipeline

# 上記の Step のみのパイプライン
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[inferenceStep])

## 2.2 パイプラインの検証

In [ ]:
pipeline.validate()
print("Pipeline validation complete")

## 2.3 パイプラインの実行

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()